In [2]:
import numpy as np
import scipy.sparse as sparse

import tqdm as tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, Normalizer, StandardScaler

import matplotlib.pyplot as plt
%matplotlib inline

seed = 69420
np.random.seed(69420)

from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.layouts import row
from bokeh.io import push_notebook

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import TfidfTransformer
from imblearn.over_sampling import SMOTE

In [2]:
Y = open('newsgrouplabels.txt', 'r')
classes = []
#print("The classes are")
for line in Y:
    y = line.split()
    classes.append(y)
    #print(y[1])

In [3]:
def read_sparse_dataset(dataset_name: str, *args, **kwargs) -> sparse.csr_matrix:
    with open(dataset_name, 'r') as f:
        num_lines = sum(1 for _ in f)

    sparse_row_list = []
    with open(dataset_name, 'r') as f:
        for row in tqdm.tqdm(f, total=num_lines):
            data = np.fromstring(row, *args, **kwargs)
            sparse_row = sparse.csr_matrix(data)
            sparse_row_list.append(sparse_row)

    data_matrix: sparse.csr_matrix
    data_matrix = sparse.vstack(sparse_row_list)
    return data_matrix

In [4]:
Xy_raw: sparse.csr_matrix
Xy_raw = read_sparse_dataset("training.csv", sep=',', dtype=np.int32)

100%|████████████████████████████████████| 12000/12000 [00:22<00:00, 527.97it/s]


In [5]:
xTest: sparse.csr_matrix
xTest = read_sparse_dataset("testing.csv", sep=',', dtype=np.int32)

100%|██████████████████████████████████████| 6774/6774 [00:13<00:00, 491.96it/s]


In [6]:
X_bow = Xy_raw[:, 1:-1].toarray()
y_bow = Xy_raw[:, -1].toarray()
xTest_bow = xTest[:, 1:].toarray()

In [7]:
transformer = TfidfTransformer()
transformer.fit(X_bow)

X_tfidf = transformer.transform(X_bow)
xTest_tfidf = transformer.transform(xTest_bow)

In [8]:
selector = SelectKBest(chi2, k=25000)
X_chi = selector.fit_transform(X_tfidf, y_bow)
xTest_chi = selector.transform(xTest_tfidf)

In [9]:
smote = SMOTE(random_state=42)
X_smote, Y_smote = smote.fit_resample(X_chi, y_bow.ravel())

In [10]:
scaler = StandardScaler(with_mean=False)
scaler.fit(X_smote)
X_norm = sparse.csr_matrix(scaler.transform(X_smote))
xTest_norm = sparse.csr_matrix(scaler.transform(xTest_chi))

In [11]:
encoder = OneHotEncoder()
y_cat = encoder.fit_transform(Y_smote.reshape(-1, 1))

In [12]:
def calculate_loss(*, y, s):
    e = 1e-8
    return -np.sum(y * np.log(s + e)).mean()

def multilogistic(*, z):
    z = z - np.max(z, axis=0, keepdims=True)
    return np.exp(z) / np.sum(np.exp(z), axis=0, keepdims=True)

def calculate_gradient(*, X, y, s):
    return (y.T - s) @ X

def predict(*, X, W):
    return multilogistic(z=W @ X.T)

def accuracy(*, y_true, y_pred):
    return np.sum(y_true == y_pred) / y_true.shape[0]

In [20]:
def logistic_regression(X, y, Xv, yv, 
                        W_init=None, epochs=1000, minEpoch=5, 
                        eta=1e-2, lambda_=1e-2, threshold=1e-6):
    if W_init is None:
        W = np.random.normal(0, 0.1, (y.shape[1], X.shape[1])) 
        W = np.hstack((np.zeros((y.shape[1],1)), W))
    else:
        W = W_init

    X = sparse.hstack((np.ones((X.shape[0], 1)), X))
    Xv = sparse.hstack((np.ones((Xv.shape[0], 1)), Xv))
    losses = []
    lossesV = []
    accuracies = []
    accuraciesV = []

#     output_notebook()

#     source_loss = ColumnDataSource(data=dict(x=[], y_train=[], y_val=[]))
#     source_acc = ColumnDataSource(data=dict(x=[], y_train=[], y_val=[]))

#     p1 = figure(title="Loss", x_axis_label='Epochs', y_axis_label='Loss', plot_width=400, plot_height=300)
#     p1.line('x', 'y_train', source=source_loss, legend_label="Train Loss", line_color="blue")
#     p1.line('x', 'y_val', source=source_loss, legend_label="Val Loss", line_color="green")

#     p2 = figure(title="Accuracy", x_axis_label='Epochs', y_axis_label='Accuracy', plot_width=400, plot_height=300)
#     p2.line('x', 'y_train', source=source_acc, legend_label="Train Accuracy", line_color="blue")
#     p2.line('x', 'y_val', source=source_acc, legend_label="Val Accuracy", line_color="green")

#     layout = row(p1, p2)
#     handle = show(layout, notebook_handle=True)

    try:
        #with epochs as t:
        for i in range(epochs):

            s = multilogistic(z=W @ X.T)  
            st = multilogistic(z=W @ Xv.T)
            gradient = calculate_gradient(X = X, y = y, s = s) - (lambda_ * W)

            loss = calculate_loss(y = y, s = s.T) + (lambda_/2) * np.sum(W**2)
            losses.append(loss)
            lossV = calculate_loss(y = yv, s = st.T) + (lambda_/2) * np.sum(W**2)
            lossesV.append(lossV)

            accuracy_ = accuracy(y_true = np.argmax(y, axis=1), y_pred = np.argmax(predict(X=X,W=W), axis=0))
            #t.set_postfix(loss=f"{loss:.6f}", accuracy=f"{accuracy_:.6f}")
            accuracies.append(accuracy_)

            accuracyV_ = accuracy(y_true = np.argmax(yv, axis=1), y_pred = np.argmax(predict(X=Xv,W=W), axis=0))
            #t.set_postfix(lossV=f"{lossV:.6f}", accuracyV=f"{accuracyV_:.6f}")
            accuraciesV.append(accuracyV_)

            #accT = 0.0001
            if i > minEpoch:
                if np.abs(losses[-1] - losses[-2]) < threshold:# or np.abs(accuracies[-1] - accuracies[-2]) < accT:
                    #print("Converged at epoch ", i)
                    break

            W = W + eta * gradient

#                 # Update plots
#                 source_loss.stream(dict(x=[i], y_train=[loss], y_val=[lossV]))
#                 source_acc.stream(dict(x=[i], y_train=[accuracy_], y_val=[accuracyV_]))

#                 # Redraw plots
#                 push_notebook(handle=handle)

    except KeyboardInterrupt:
        print("Interrupted by user at epoch ", i)

    return W, losses, lossesV, accuracies, accuraciesV


In [14]:
X_train_chi, X_val_chi, y_train_chi, y_val_chi = train_test_split(X_norm, y_cat, test_size=0.2, stratify=np.asarray(np.argmax(y_cat, axis=1)), random_state=seed) # type: ignore
y_train_chi, y_val_chi = y_train_chi.toarray(), y_val_chi.toarray() # type: ignore

In [16]:
WvChi, lossesvChi, lossesVvChi, accuraciesChi, accuraciesVChi = logistic_regression(X_train_chi, y_train_chi, X_val_chi, y_val_chi, epochs = 1000, minEpoch = 5, eta = 1e-1, lambda_ = 1e-3, threshold = 1)

Converged at epoch  8


In [17]:
X_val_chi_p = sparse.hstack((np.ones((X_val_chi.shape[0], 1)), X_val_chi))
st = predict(X = X_val_chi_p, W = WvChi)

In [18]:
y_true = np.argmax(y_val_chi, axis=1)  # Ground truth labels
y_pred = np.argmax(st, axis=0)  # Predicted labels
labels = list(np.array(classes)[:, 1])  # List of unique class labels

print(classification_report(y_true, y_pred, target_names=labels))

                          precision    recall  f1-score   support

             alt.atheism       0.86      0.95      0.91       130
           comp.graphics       0.94      0.78      0.85       130
 comp.os.ms-windows.misc       0.88      0.87      0.88       130
comp.sys.ibm.pc.hardware       0.84      0.75      0.79       131
   comp.sys.mac.hardware       0.81      0.85      0.83       130
          comp.windows.x       0.92      0.88      0.90       130
            misc.forsale       0.92      0.83      0.87       130
               rec.autos       0.99      0.88      0.93       130
         rec.motorcycles       0.93      0.96      0.94       130
      rec.sport.baseball       0.97      0.97      0.97       130
        rec.sport.hockey       0.98      0.95      0.96       130
               sci.crypt       0.94      0.98      0.96       130
         sci.electronics       0.85      0.88      0.86       130
                 sci.med       0.89      0.95      0.92       130
         

In [ ]:
import pandas as pd
xTest_norm_p = sparse.hstack((np.ones((xTest_norm.shape[0], 1)), xTest_norm))
yPred = np.argmax(predict(W = WvChi, X = xTest_norm_p), axis=0) + 1

In [ ]:
predictions = pd.DataFrame(pd.DataFrame(yPred))
predictions.columns = ["class"]
predictions.index = np.arange(1, len(predictions) + 1)
predictions.set_index(pd.Index(range(12001, 12001 + len(predictions))), inplace=True)
predictions.index.name='id'
predictions.to_csv("predicions.csv")

In [21]:
# Define the range of values for eta and lambda
eta_values = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
lambda_values = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]

# Initialize lists to store the results
train_accuracies = []
val_accuracies = []
train_losses = []
val_losses = []

# Loop through all combinations of eta and lambda and perform logistic regression for each combination
for eta in eta_values:
    for lambda_ in lambda_values:
        # Train the logistic regression model
        WvChi, lossesvChi, lossesVvChi, accuraciesChi, accuraciesVChi = logistic_regression(X_train_chi, y_train_chi, X_val_chi, y_val_chi, epochs=1000, minEpoch=5, eta=eta, lambda_=lambda_, threshold=1)
        
        X_train_chi_p = sparse.hstack((np.ones((X_train_chi.shape[0], 1)), X_train_chi))
        s = predict(X = X_train_chi_p, W = WvChi)
        # Calculate the accuracy and loss on the training set
        train_accuracy = accuracy(y_true=np.argmax(y_train_chi, axis=1), y_pred=np.argmax(s, axis=0))
        train_loss = calculate_loss(y=y_train_chi, s=s.T)
        
        X_val_chi_p = sparse.hstack((np.ones((X_val_chi.shape[0], 1)), X_val_chi))
        st = predict(X = X_val_chi_p, W = WvChi)
        # Calculate the accuracy and loss on the validation set
        val_accuracy = accuracy(y_true=np.argmax(y_val_chi, axis=1), y_pred=np.argmax(st, axis=0))
        val_loss = calculate_loss(y=y_val_chi, s=st.T)

        # Append the results to the lists
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        print(f"eta {eta}, lambda {lambda_}, Train Accuracy {train_accuracy}, Validation Accuracy {val_accuracy}, Train Loss {train_loss}, Validation Loss {val_loss}")

# Reshape the lists into matrices with one dimension for each hyperparameter
train_accuracies = np.array(train_accuracies).reshape(len(eta_values), len(lambda_values))
val_accuracies = np.array(val_accuracies).reshape(len(eta_values), len(lambda_values))
train_losses = np.array(train_losses).reshape(len(eta_values), len(lambda_values))
val_losses = np.array(val_losses).reshape(len(eta_values), len(lambda_values))

# Find the optimal values of eta and lambda that give the best validation accuracy
best_eta_index, best_lambda_index = np.unravel_index(np.argmax(val_accuracies), val_accuracies.shape)
best_eta = eta_values[best_eta_index]
best_lambda = lambda_values[best_lambda_index]

print(f"Best eta: {best_eta}")
print(f"Best lambda: {best_lambda}")

eta 1e-05, lambda 1e-05, Train Accuracy 0.9987519201228878, Validation Accuracy 0.5061443932411674, Train Loss 111.73329150534803, Validation Loss 15545.080277813255
eta 1e-05, lambda 0.0001, Train Accuracy 0.9989439324116743, Validation Accuracy 0.5372503840245776, Train Loss 98.27532192031745, Validation Loss 14917.913094484617
eta 1e-05, lambda 0.001, Train Accuracy 0.9987519201228878, Validation Accuracy 0.5272657450076805, Train Loss 130.34241761730985, Validation Loss 14784.171416562811
eta 1e-05, lambda 0.01, Train Accuracy 0.9988479262672811, Validation Accuracy 0.5084485407066052, Train Loss 103.9424162041716, Validation Loss 15620.053452084703
eta 1e-05, lambda 0.1, Train Accuracy 0.9986559139784946, Validation Accuracy 0.5249615975422427, Train Loss 125.56923422919087, Validation Loss 15061.99201473766
eta 1e-05, lambda 1, Train Accuracy 0.9986559139784946, Validation Accuracy 0.5264976958525346, Train Loss 119.39444092120259, Validation Loss 15156.67636380321
eta 0.0001, la